In [1]:
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

/tmp/ipykernel_661958/4081799432.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_5 = pd.read_csv('./in/copper_returns_5d_final.csv')
df_10 = pd.read_csv('./in/copper_returns_10d_final.csv')
df_20 = pd.read_csv('./in/copper_returns_20d_final.csv')

print(df_5)
print(df_10)
print(df_20)

            Date  LMCADS03    LMCADY       DXY       SPX      BCOM      MXWD  \
0     2024-03-25  0.001241  0.001047 -0.000767  0.002808  0.006127  0.001182   
1     2024-03-22 -0.000733 -0.000182  0.002015  0.003064 -0.005829  0.002399   
2     2024-03-21  0.009474  0.009322 -0.004022  0.001404  0.007491  0.002585   
3     2024-03-20 -0.002514 -0.002130 -0.005961 -0.003226 -0.001953 -0.006740   
4     2024-03-19  0.005432  0.005893  0.004159 -0.008826  0.003441 -0.006092   
...          ...       ...       ...       ...       ...       ...       ...   
5534  2003-01-07 -0.009774 -0.009724  0.006872  0.014287 -0.004085  0.014765   
5535  2003-01-06  0.004627  0.004489 -0.005850  0.006588  0.002701  0.008892   
5536  2003-01-03 -0.010746 -0.010210  0.004903 -0.021980  0.016248 -0.018378   
5537  2003-01-02 -0.013035 -0.013388  0.004977  0.000484 -0.014186 -0.003775   
5538  2003-01-01 -0.018868 -0.019639 -0.010682 -0.032133 -0.025019 -0.022941   

           XAU       XAG  LMCADY_acu_5d

In [3]:
df_5['Date'] = pd.to_datetime(df_5['Date'])
df_5.set_index('Date', inplace=True)

In [4]:
target = 'LMCADY_acu_5d'
train_data, test_data = train_test_split(df_5[target], test_size=0.2, shuffle=False)

In [5]:
model_ar = AutoReg(train_data, lags=5, old_names=False)
model_ar_fitted = model_ar.fit()

/home/fochoa/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/fochoa/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [6]:
predictions = model_ar_fitted.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1, dynamic=False)

/home/fochoa/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/fochoa/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/home/fochoa/.local/lib/python3.10/site-packages/statsmodels/tsa/deterministic.py:307: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


In [7]:
mse = mean_squared_error(test_data, predictions)
rmse = np.sqrt(mse)

In [8]:
model_ar_fitted.summary(), rmse

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             AutoReg Model Results                             
 Dep. Variable:          LMCADY_acu_5d   No. Observations:                 4431
 Model:                     AutoReg(5)   Log Likelihood               10902.522
 Method:               Conditional MLE   S.D. of innovations              0.021
 Date:                Tue, 16 Apr 2024   AIC                         -21791.044
 Time:                        17:59:28   BIC                         -21746.277
 Sample:                             5   HQIC                        -21775.257
                                  4431                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
 ------------------------------------------------------------------------------------
 const                0.0001      0.000      0.470      0.638      -0.000       0.001
 LMCADY_acu_5d.L1     0.7672      0.015     52.193  